<a href="https://colab.research.google.com/github/CallmeKoi/DeepL-Semantic-Segment/blob/main/DoAn_Unet_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lấy dataset từ repo trên github

In [1]:
!git clone https://github.com/CallmeKoi/DeepL-Semantic-Segment.git

fatal: destination path 'DeepL-Semantic-Segment' already exists and is not an empty directory.


In [3]:
# @title Thêm đường dẫn tới repo để dùng custom module
# Cách này bị giới hạn trong phiên hiện tại
import sys

# append a new directory to sys.path
sys.path.append('/content/DeepL-Semantic-Segment')

# print the updated sys.path
print('Updated sys.path:', sys.path)

Updated sys.path: ['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/DeepL-Semantic-Segment']


In [4]:
# @title Import thư viện
import os
import pandas as pd
import numpy as np
from matplotlib import pylab as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Import custom module
from dataset_util import DataLoader, preprocessing
from models.Unet_model import Unet

In [4]:
# @title File metadata.csv chứa một số thông tin liên quan tới dataset
metadata = pd.read_csv('/content/DeepL-Semantic-Segment/metadata.csv')
print(len(metadata))
metadata.groupby(['Purpose'])['Class'].value_counts()


1000


Purpose  Class     
test     bicycle        50
         car            50
         motorcycle     50
         pedestrian     50
train    bicycle       150
         car           150
         motorcycle    150
         pedestrian    150
valid    bicycle        50
         car            50
         motorcycle     50
         pedestrian     50
Name: Class, dtype: int64

In [5]:
# @title File class_label.csv chứa nhãn lớp
class_df = pd.read_csv('/content/DeepL-Semantic-Segment/class_label.csv')
class_label = pd.Series(class_df['class'].values,index=class_df.label).to_dict()
print(class_label)

{0: 'background', 1: 'car', 2: 'bicycle', 3: 'motorcycle', 4: 'pedestrian'}


### Chuẩn bị dữ liệu

In [6]:
# Lấy dataset
dir_to_data = '/content/DeepL-Semantic-Segment/zoo_dataset'
dataloader = DataLoader(dir_to_data)

# Load phần ảnh cần dùng
x_train, y_train = dataloader.load('train',(224,224),shuffle=True)
x_valid, y_valid = dataloader.load('valid',(224,224),shuffle=True)

print('Train data shape:',x_train.shape)
print('Train label shape:',y_train.shape)
print('Valid data shape:',x_valid.shape)
print('Valid label shape:',y_valid.shape)

# Tiền xử lý ảnh
NCLASSES = len(class_label) # số lượng lớp
X_train, Y_train = preprocessing(x_train, y_train, NCLASSES)
X_valid, Y_valid = preprocessing(x_valid, y_valid, NCLASSES)


('/content/DeepL-Semantic-Segment/zoo_dataset/train/00399.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00590.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00594.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00303.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00016.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00634.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00878.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00602.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00873.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00635.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00050.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00764.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00849.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00309.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/00808.jpg', '/content/DeepL-Semantic-Segment/zoo_dataset/train/005

### Huấn luyện mô hình Unet cổ điển

In [7]:
# Load model
input_shape = X_train.shape[1:]
'''
Mô hình Unet nhận vào các tham số bắt buộc: input_shape, output_classes
Tham số tùy chỉnh: dropout_rate
'''
model = Unet(input_shape,NCLASSES, dropout_rate=0.3)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Main Input (InputLayer)     [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 En_1_conv1 (Conv2D)         (None, 224, 224, 64)         1792      ['Main Input[0][0]']          
                                                                                                  
 En_1_norm1 (BatchNormaliza  (None, 224, 224, 64)         256       ['En_1_conv1[0][0]']          
 tion)                                                                                            
                                                                                                  
 En_1_act1 (Activation)      (None, 224, 224, 64)         0         ['En_1_norm1[0][0]']      

In [9]:
# Chuyển dữ liệu về format tf.data

def tf_data(X_train, Y_train, X_val, Y_val, batch_size):
    train_ds = tf.data.Dataset.from_tensor_slices((X_train,
                                                   Y_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    val_ds = tf.data.Dataset.from_tensor_slices((X_val,
                                                 Y_val)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return train_ds, val_ds

In [10]:
# Tinh chỉnh siêu tham số
batch_size = 16
epochs = 20

# Callback
cb = [EarlyStopping(monitor='val_loss', patience=5,
                    restore_best_weights=True)]

train_ds, val_ds = tf_data(X_train, Y_train, X_valid, Y_valid, batch_size)

H = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs, callbacks = cb
)

# Save model
save_dir = os.path.join(os.getcwd(), 'trained_models')
if (not os.path.exists(save_dir)):
  os.makedirs(save_dir)

save_path = os.path.join(save_dir,'unet.best_model.keras')
model.save(save_path)

Epoch 1/20
38/38 [==============================] - 100s 1s/step - loss: 1.1900 - accuracy: 0.7286 - val_loss: 34216.3984 - val_accuracy: 0.0720
Epoch 2/20
38/38 [==============================] - 35s 915ms/step - loss: 0.5894 - accuracy: 0.8682 - val_loss: 0.8305 - val_accuracy: 0.8160
Epoch 3/20
38/38 [==============================] - 35s 930ms/step - loss: 0.5268 - accuracy: 0.8696 - val_loss: 0.6895 - val_accuracy: 0.8558
Epoch 4/20
38/38 [==============================] - 36s 938ms/step - loss: 0.5123 - accuracy: 0.8698 - val_loss: 0.5986 - val_accuracy: 0.8558
Epoch 5/20
38/38 [==============================] - 36s 942ms/step - loss: 0.4996 - accuracy: 0.8696 - val_loss: 0.5920 - val_accuracy: 0.8558
Epoch 6/20
38/38 [==============================] - 36s 953ms/step - loss: 0.4903 - accuracy: 0.8701 - val_loss: 0.5752 - val_accuracy: 0.8558
Epoch 7/20
38/38 [==============================] - 36s 957ms/step - loss: 0.4792 - accuracy: 0.8702 - val_loss: 0.5647 - val_accuracy: 0.85

In [1]:
# @title In đường cong học tập
print(H.history.keys())
plt.figure()
pd.DataFrame(H.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
# plt.plot(H.history['accuracy'], label="train_acc")
# plt.plot(H.history['val_accuracy'], label="val_acc")
# plt.plot(H.history['loss'], label = "train_loss")
# plt.plot(H.history['val_loss'], label = "val_loss")
plt.title('Training Loss and Accuracy')
plt.ylabel('Loss/Accuracy')
plt.xlabel('Epoch #')
plt.legend()
plt.show()

NameError: name 'H' is not defined